In [1]:
spark

# Load Iris Data

In [5]:
from sklearn import datasets
iris = datasets.load_iris()

In [50]:
X = pd.DataFrame(iris['data'],columns=iris['feature_names'])
y = pd.DataFrame(iris['target'],columns=['target_names'])
data = pd.concat([X,y],axis=1)
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [16]:
import numpy as np
# 加上隨機的 group_id
group_id = pd.DataFrame(np.random.randint(0,3, len(data)), columns=['group_id'])
grouped_data = pd.concat([data,group_id],axis=1)
grouped_data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target_names,group_id
0,5.1,3.5,1.4,0.2,0,0
1,4.9,3.0,1.4,0.2,0,0
2,4.7,3.2,1.3,0.2,0,0
3,4.6,3.1,1.5,0.2,0,0
4,5.0,3.6,1.4,0.2,0,2
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,2
146,6.3,2.5,5.0,1.9,2,1
147,6.5,3.0,5.2,2.0,2,0
148,6.2,3.4,5.4,2.3,2,1


In [58]:
df = spark.createDataFrame(grouped_data)
df.show()

+-----------------+----------------+-----------------+----------------+------------+--------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target_names|group_id|
+-----------------+----------------+-----------------+----------------+------------+--------+
|              5.1|             3.5|              1.4|             0.2|           0|       0|
|              4.9|             3.0|              1.4|             0.2|           0|       0|
|              4.7|             3.2|              1.3|             0.2|           0|       0|
|              4.6|             3.1|              1.5|             0.2|           0|       0|
|              5.0|             3.6|              1.4|             0.2|           0|       2|
|              5.4|             3.9|              1.7|             0.4|           0|       0|
|              4.6|             3.4|              1.4|             0.3|           0|       1|
|              5.0|             3.4|              1.5|      

# pandas_udf sklearn

In [19]:
from pyspark.sql.types import *
 
# define schema for what the pandas udf will return
schema = StructType([
StructField('group_id', IntegerType()),
StructField('num_instances_trained_with', IntegerType()),
StructField('model_str', StringType())
])

In [20]:
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType

In [21]:
@pandas_udf(schema, functionType=PandasUDFType.GROUPED_MAP)

def train_model(df_pandas):
    '''
    Trains a RandomForestRegressor model on training instances
    in df_pandas.
 
    Assumes: df_pandas has the columns:
                 ['my_feature_1', 'my_feature_2', 'my_label']
 
    Returns: a single row pandas DataFrame with columns:
               ['group_id', 'num_instances_trained_with', 
                'model_str']
    '''
 
    # get the value of this group id
    group_id = df_pandas['group_id'].iloc[0]
 
    # get the number of training instances for this group
    num_instances = df_pandas.shape[0]
 
    # get features and label for all training instances in this group
    feature_columns = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
    label = 'target_names';
    X = df_pandas[feature_columns]
    Y = df_pandas[label]
 
    # train this model
    model = RandomForestRegressor()
    model.fit(X,Y)
 
    # get a string representation of our trained model to store
    model_str = pickle.dumps(model)
 
    # build the DataFrame to return
    df_to_return = pd.DataFrame([group_id, num_instances, model_str],
    columns = ['group_id', 'num_instances_trained_with', 'model_str'])
 
    return df_to_return

In [62]:
df_trained_models = df.groupBy('group_id').apply(train_model)

In [66]:
df_trained_models.toPandas().show()

22/02/14 03:36:08 ERROR Executor: Exception in task 0.0 in stage 20.0 (TID 20)1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_2602/4169914161.py", line 36, in train_model
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py", line 738, in __init__
    mgr = ndarray_to_mgr(
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/internals/construction.py", line 351, in ndarray_to_mgr
    _check_values_indices_shape_match(values, index, columns)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/internals/construction.py", line 422, in _check_values_indices_shape_match
    raise ValueError(f"Shape of passed values is {passed}, indices imply {implied}")
ValueError: Shape of passed values is (3, 1), indices imply (3, 3)

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:555)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(Python

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_2602/4169914161.py", line 36, in train_model
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py", line 738, in __init__
    mgr = ndarray_to_mgr(
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/internals/construction.py", line 351, in ndarray_to_mgr
    _check_values_indices_shape_match(values, index, columns)
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/internals/construction.py", line 422, in _check_values_indices_shape_match
    raise ValueError(f"Shape of passed values is {passed}, indices imply {implied}")
ValueError: Shape of passed values is (3, 1), indices imply (3, 3)


In [65]:
import pickle
pickle.load(df_trained_models['model_str'])

TypeError: 'Column' object is not callable